In [12]:
import pandas as pd
import nltk
import numpy as np
import re
import random
from nltk.stem import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import pos_tag
from sklearn.metrics import pairwise_distances
from nltk import word_tokenize
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

In [13]:
def text_normalization(text):
    text=str(text).lower()
    special_char_text=re.sub(r'[^a-z]',' ',text)
    tokenized_text=nltk.word_tokenize(special_char_text)
    lemmatized_text=wordnet.WordNetLemmatizer()
    tags_list=pos_tag(tokenized_text,tagset=None)
    lema_words=[]
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lemmatized_text.lemmatize(token,pos_val)
        lema_words.append(lema_token)
        
    return " ".join(lema_words)
    

In [14]:
def splitting_of_Input(u_input):
    q=[]
    a=u_input.split()
    for i in a:
        if i in stop:
            continue
        else:
            q.append(i)
        b=" ".join(q)
    global ques_lema
    global ques_bow
    ques_lema=text_normalization(b)
    ques_bow=cv.transform([ques_lema]).toarray() 

In [15]:
def chat_tfidf(text):
    lemma=text_normalization(text)
    tf=tfidf.transform([lemma]).toarray()
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine')
    index_value=cos.argmax()
    return [df['Remedies'].loc[index_value]]

In [16]:
Greeting_Inputs = ("hello", "hi", "greetings", "sup", "what's up", "hey", "Could you help me")
Greeting_Responses = ["Hi. I will assist you.", "Hey. I will assist you.", "*nods*", "Hi there. I will assist you.", "Hello. I will assist you.", "I am glad you are talking to me. I will assist you."]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in Greeting_Inputs:
            return random.choice(Greeting_Responses) 

In [17]:
df=pd.read_excel("remibot_dataset_1614271001.xlsx")

In [18]:
df['lemmatized_symptoms']=df['Symptom'].apply(text_normalization)

In [9]:
stop=stopwords.words('english')
stop_words = ['?',',','.','!','(',')']
for i in stop_words:
    stop.append(i)

In [19]:
continue_diag = True
print("Remibot: Hello I am Remibot! You can Type the symptoms that you are feeling.")

while(continue_diag==True):
    user_input = input("User:")
    ques = user_input.lower()
      
    cv=CountVectorizer()
    x=cv.fit_transform(df['lemmatized_symptoms']).toarray()
    
    splitting_of_Input(ques)
    
    tfidf=TfidfVectorizer()
    x_tfidf=tfidf.fit_transform(df['lemmatized_symptoms']).toarray()
    
    df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names_out())
    ques_tfidf=tfidf.transform([ques_lema]).toarray()
  
    
    features=cv.get_feature_names_out()
    df_bow=pd.DataFrame(x,columns=features)

    cosine_value=1-pairwise_distances(df_bow,ques_bow,metric='cosine')
    index_value=cosine_value.argmax()
    if ques != "bye":
        if ques == "thanks" or ques == "thank you" or ques == "thank you so much" or ques == "thank you very much":
            continue_diag = False
            print("Remibot: Most Welcome")
        else:
            if greeting(ques) != None:
                print(f"Remibot: {greeting(ques)}")
            else:
                for k in chat_tfidf(ques):
                    var_n = (f"The Natural Remedies for the following Symptoms are:\n {k}")   
                print(f"Remibot: {var_n}")
    else:
        continue_diag = False
        print("Bye. Take care...")

Remibot: Hello I am Remibot! You can Type the symptoms that you are feeling.


User: cough


Remibot: The Natural Remedies for the following Symptoms are:
 Caffeinated tea or coffee,  Lavender oil


User: cold, cough, fever


Remibot: The Natural Remedies for the following Symptoms are:
 ginger, garlic, turmeric


User: bye


Bye. Take care...
